## 1. Create Entropy

In [4]:
import secrets

# Define strength (in bits) for the entropy
strength = 128  # You can set this to 128, 160, 192, 224, or 256
strength_bytes = strength // 8

# Generate entropy using the secrets module
entropy = secrets.token_bytes(strength_bytes)

# Print the generated entropy in hexadecimal format
print(f'Created entropy: {entropy.hex()}')

Created entropy: 9ad4696b163ba9dbc31e663ae733006b


# Create Mnemonic from Entropy

## 2. Create Mnemonic
### 2-1. Load wordlist

In [10]:
wordlist = []
with open("english.txt", "r", encoding="utf-8") as f:
    wordlist = [w.strip() for w in f.readlines()]

print('Successfully loaded wordlist')
wordlist

Successfully loaded wordlist


['abandon',
 'ability',
 'able',
 'about',
 'above',
 'absent',
 'absorb',
 'abstract',
 'absurd',
 'abuse',
 'access',
 'accident',
 'account',
 'accuse',
 'achieve',
 'acid',
 'acoustic',
 'acquire',
 'across',
 'act',
 'action',
 'actor',
 'actress',
 'actual',
 'adapt',
 'add',
 'addict',
 'address',
 'adjust',
 'admit',
 'adult',
 'advance',
 'advice',
 'aerobic',
 'affair',
 'afford',
 'afraid',
 'again',
 'age',
 'agent',
 'agree',
 'ahead',
 'aim',
 'air',
 'airport',
 'aisle',
 'alarm',
 'album',
 'alcohol',
 'alert',
 'alien',
 'all',
 'alley',
 'allow',
 'almost',
 'alone',
 'alpha',
 'already',
 'also',
 'alter',
 'always',
 'amateur',
 'amazing',
 'among',
 'amount',
 'amused',
 'analyst',
 'anchor',
 'ancient',
 'anger',
 'angle',
 'angry',
 'animal',
 'ankle',
 'announce',
 'annual',
 'another',
 'answer',
 'antenna',
 'antique',
 'anxiety',
 'any',
 'apart',
 'apology',
 'appear',
 'apple',
 'approve',
 'april',
 'arch',
 'arctic',
 'area',
 'arena',
 'argue',
 'arm',
 

### 2-2. Calculate hash of entropy

In [13]:
import hashlib

h = hashlib.sha256(entropy).hexdigest()
print(f'Calculated sha256 hash of entropy: {h}')

Calculated sha256 hash of entropy: 252187405b14b01bb16a31419ce538a28bc320cb1b5a35c892ac347ad032e824


### 2-3. Create entropy + checksum bits for creating Mnemonic

In [24]:
entropy_bits = b = bin(int.from_bytes(entropy, byteorder="big"))[2:].zfill(len(entropy) * 8)
print(f'Entroy bits: {entropy_bits} (len: {len(entropy_bits)})')
checksum_bits = bin(int(h, 16))[2:].zfill(256)[: len(entropy) * 8 // 32]
print(f'Checksum bits: {checksum_bits} (len: {len(checksum_bits)})')

bits = entropy_bits + checksum_bits
print(f'Bits: {bits} (len: {len(bits)})')

Entroy bits: 10011010110101000110100101101011000101100011101110101001110110111100001100011110011001100011101011100111001100110000000001101011 (len: 128)
Checksum bits: 0010 (len: 4)
Bits: 100110101101010001101001011010110001011000111011101010011101101111000011000111100110011000111010111001110011001100000000011010110010 (len: 132)


### 2-4. Create mnemonic from entropy + checksum bits 

- Split the combined bitstring: The code takes the entropy (random data) and appends a checksum calculated from its hash (SHA-256). This combined bitstring is then split into fixed-length 11-bit segments.

- Convert binary to integer: Each 11-bit segment is interpreted as a binary number and converted to an integer using int(segment, 2). This integer serves as an index for the BIP-39 wordlist.

- Map integers to words: The integers derived from each 11-bit segment are used to select words from the predefined BIP-39 wordlist. These words collectively form the mnemonic phrase.

- Construct the mnemonic phrase: The selected words are appended to form a human-readable and secure mnemonic phrase, which can later be used to derive cryptographic keys.

In [29]:
result = []
for i in range(len(bits) // 11):
    idx = int(bits[i * 11 : (i + 1) * 11], 2)
    print(f'Calculated idx: {idx}')
    word = wordlist[idx]
    print(f'Word at {idx}: {word}')
    result.append(word)

print(f'Successfully created mnemonic: {result}')

Calculated idx: 1238
Word at 1238: onion
Calculated idx: 1306
Word at 1306: person
Calculated idx: 726
Word at 726: food
Calculated idx: 355
Word at 355: cluster
Calculated idx: 1492
Word at 1492: ritual
Calculated idx: 1903
Word at 1903: unknown
Calculated idx: 99
Word at 99: arrest
Calculated idx: 1638
Word at 1638: smoke
Calculated idx: 471
Word at 471: depend
Calculated idx: 460
Word at 460: define
Calculated idx: 1536
Word at 1536: scale
Calculated idx: 1714
Word at 1714: stone
Successfully created mnemonic: ['onion', 'person', 'food', 'cluster', 'ritual', 'unknown', 'arrest', 'smoke', 'depend', 'define', 'scale', 'stone']
